# Data Acquisition 

The very first step of any computational experiment is always obtaining data. Most of the times this is the very tedious part and it pays off to spend a lot of time understanding, cleaning and refining your data set. For the sake of simplicity, we will take a few shortcuts in this tutorial, but we will try to hint at possibilities to improve the process as we go on.

## Papyrus Data Set

The [Papyrus](https://chemrxiv.org/engage/chemrxiv/article-details/617aa2467a002162403d71f0) is a curated data set that tries to make our lives easier by agregating publically available data under one roof and already does a few things for us, such as standardizing the compounds, aggregating and transforming activity values and giving our data overall structure that is suitable for data analysis and machine learning. However, there are also downsides to using such data set. For example, the fact that this is a curated data set means that it needs to be updated each time new information becomes available so you might not always have the most recent data to work with. In addition, sometimes the form of standardization of the data set might not be suitable for your case so you still need to spend some time reading about how the data is curated and if valuable information is not lost. For example, Papyrus does not include stereochemistry by default so if you want that kind of data (i.e. for docking), you have to retrieve it seperately.

In this tutorial, we will be using a tool (`papyrus-scaffold-visualizer`) that enables easy extraction of data from Papyrus, but also has some simple chemical space depiction features. First, we need UniProt accession keys of our protein of interest. For the purpose of this tutorial, it will be the [Adenosine A2A receptor](https://www.uniprot.org/uniprotkb/P29274/entry):

---
*For the purpose of our ACMDD project, the protein of interest is changed to  BACE1* 

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# define work directory to store data
DATA_ROOT = '/content/drive/MyDrive/DrugExDemo/' # or wherever you want the generated files to live on your GoogleDrive
import os
os.makedirs(DATA_ROOT, exist_ok=True)
os.chdir(DATA_ROOT) 

# fetch pretrained model
os.makedirs("./data/drugex/models/pretrained/", exist_ok=True)
! wget -nc -P './data/drugex/models/pretrained/' 'https://zenodo.org/record/7096859/files/DrugEx_v2_PT_Papyrus05.5.zip'
! unzip -n './data/drugex/models/pretrained/DrugEx_v2_PT_Papyrus05.5.zip' -d './data/drugex/models/pretrained/DrugEx_v2_PT_Papyrus05.5'

# install dependencies
! git clone https://github.com/martin-sicho/drugex-demo
! pip install -r drugex-demo/requirements.txt

# verify where we are working
os.getcwd()

Mounted at /content/drive
File ‘./data/drugex/models/pretrained/DrugEx_v2_PT_Papyrus05.5.zip’ already there; not retrieving.

Archive:  ./data/drugex/models/pretrained/DrugEx_v2_PT_Papyrus05.5.zip
fatal: destination path 'drugex-demo' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/martin-sicho/DrugEx-CDDG.git (to revision master) to /tmp/pip-install-7g3pv_e4/drugex_10dc561fb44546978ac2a6e75a15c611
  Running command git clone -q https://github.com/martin-sicho/DrugEx-CDDG.git /tmp/pip-install-7g3pv_e4/drugex_10dc561fb44546978ac2a6e75a15c611
  Cloning https://github.com/martin-sicho/papyrus-scaffold-visualizer.git (to revision v0.2.0) to /tmp/pip-install-7g3pv_e4/papyrus-scaffold-visualizer_c26443da64e143318d8db32185f3d2d3
  Running command git clone -q https://github.com/martin-sicho/papyrus-scaffold-visualizer.git /tmp/pip-install-7g3pv_e4/papyrus-scaffold-visu

'/content/drive/MyDrive/DrugExDemo'

In [ ]:
acc_keys = ["P56817"] # BACE1 (https://www.uniprot.org/uniprotkb/P56817/entry)
name = "BACE_LIGANDS" # name of the file to be generated
quality = "low" # choose minimum quality from {"high", "medium", "low"}

These will be our basic settings for the data extraction from Papyrus. In short, we will get compound structures and their measured activities on the BACE  that should be of `low` quality or higher as specified by Papyrus' data curation workflow. For the purpose of this tutorial, we are interested in all available data (`low` and higher quality), but in some other applications you might want a more stringent filter. We suggest you read the documentation of Papyrus to learn how these quality classes are determined.

The `Papyrus` class exposes some basic configuration features with the help of the more powerful [`papyrus_scripts`](https://github.com/OlivierBeq/Papyrus-scripts) package. It will fetch the latest available version of Papyrus data automatically for us as as specified. Here, we are asking only for the latest Papyrus data files without definitions of stereochemistry and all data will be placed in the `./data` directory:

In [ ]:
from scaffviz.data.papyrus import Papyrus

# fetches latest version of Papyrus if not already available and filters out BACE data for us
papyrus = Papyrus(
    data_dir="./data", # data storage directory 
    stereo=False, # do not include stereochemistry
    version='latest'
)

No data was transferred yet, but once we attempt to filter for our data, the data should start downloading:

In [ ]:
dataset = papyrus.getData(
    acc_keys,
    quality,
    name,
    use_existing=True # use existing data set if it was already compiled before to speed things up
)

Latest version: 05.5
Number of files to be donwloaded: 6
Total size: 118MB


Donwloading version 05.5:   0%|          | 0.00/118M [00:00<?, ?B/s]

Using existing data from ./data/BACE_LIGANDS.tsv...


The Papyrus data will only be fetched once if the latest version is already available in the `./data` folder. Plus, thanks to the `use_existing` parameter the created data set file will also be reused once extracted. You can find it here:

In [ ]:
dataset.path

'./data/BACE_LIGANDS.tsv'

It is no special file and you can either use `pandas` to load it manually into a `DataFrame` in your environment or get it sraight from the instance returned by the `getData()` method:

In [ ]:
dataset.asDataFrame().head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,Descriptor_MorganFP_1016,Descriptor_MorganFP_1017,Descriptor_MorganFP_1018,Descriptor_MorganFP_1019,Descriptor_MorganFP_1020,Descriptor_MorganFP_1021,Descriptor_MorganFP_1022,Descriptor_MorganFP_1023,TSNE_1,TSNE_2
0,AAAGFCUDAXPDAA_on_P56817_WT,High,ChEMBL30,CHEMBL1915983,COCC1CCN(C(C)C(=O)NC(Cc2cc(F)cc(F)c2)C(O)CNCc2...,AAAGFCUDAXPDAA,AAAGFCUDAXPDAA-UHFFFAOYSA-N,InChI=1S/C27H35F2N3O5/c1-17(32-8-7-20(16-36-2)...,"""AuxInfo=1/1/N:9,1,32,34,35,33,5,6,16,22,29,14...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-61.671165,18.636885
1,AACUJFVOHGRMTR_on_P56817_WT,High,ChEMBL30,CHEMBL4204869,CC1OCC2(c3cc(NC(=O)c4cnc(C(F)F)cn4)ccc3F)N=C(N...,AACUJFVOHGRMTR,AACUJFVOHGRMTR-UHFFFAOYSA-N,InChI=1S/C19H18F3N5O2S/c1-9-12-7-30-18(23)27-1...,"""AuxInfo=1/1/N:1,21,22,7,19,13,29,4,2,8,6,30,2...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.981135,-41.491680
2,AANLQBIDWIHDGD_on_P56817_WT,High,ChEMBL30,CHEMBL4159574,COCCOc1cnc(C(=O)Nc2ccc(F)c(C3(C)N=C(N)COC3C(F)...,AANLQBIDWIHDGD,AANLQBIDWIHDGD-UHFFFAOYSA-N,"InChI=1S/C20H21F4N5O4/c1-19(18(20(22,23)24)33-...","""AuxInfo=1/1/N:20,1,14,15,3,4,31,32,7,24,13,18...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,22.383297,-54.882477
3,AARJXZHQDZOZNY_on_P56817_WT,High,ChEMBL30,CHEMBL2443370,CC1=C(C(=O)Nc2nc(C)cs2)C(c2ccco2)C(C(=O)Nc2nc(...,AARJXZHQDZOZNY,AARJXZHQDZOZNY-UHFFFAOYSA-N,InChI=1S/C21H21N5O3S2/c1-10-8-30-20(22-10)25-1...,"""AuxInfo=1/1/N:10,26,1,30,16,15,17,11,27,9,25,...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,26.984573,25.672077
4,AAWSEFDIDJLXJF_on_P56817_WT,High,ChEMBL30,CHEMBL2178717,CN1C(=O)C2CN(c3ncccn3)CC2(c2cc(-c3cccc(C#N)c3)...,AAWSEFDIDJLXJF,AAWSEFDIDJLXJF-UHFFFAOYSA-N,InChI=1S/C22H19N7OS/c1-28-19(30)17-11-29(21-25...,"""AuxInfo=1/1/N:1,21,11,22,20,10,12,26,17,24,6,...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-53.715622,-27.205227


This is a table you can already work with, but the `dataset` object is an instance of the `scaffviz.data.dataset.DataSetTSV` class that supports some more cheminformatics features that might be of interest: 

In [ ]:
type(dataset)

scaffviz.data.dataset.DataSetTSV

For example, we can easily attach predefined or our own molecular descriptors if we need to (however, note that `Papyrus` class itself also allows you to define descriptors to fetch with the data):

In [ ]:
from scaffviz.clustering.descriptors import MorganFP

# add Morgan fingerprints to the data set -> will be used to calculate the t-SNE embedding for chemical space visualization
dataset.addDescriptors([MorganFP(radius=2, nBits=1024)], recalculate=False) 
dataset.getDescriptors()

,Descriptor_MorganFP_0,Descriptor_MorganFP_1,Descriptor_MorganFP_2,Descriptor_MorganFP_3,Descriptor_MorganFP_4,Descriptor_MorganFP_5,Descriptor_MorganFP_6,Descriptor_MorganFP_7,Descriptor_MorganFP_8,Descriptor_MorganFP_9,...,Descriptor_MorganFP_1014,Descriptor_MorganFP_1015,Descriptor_MorganFP_1016,Descriptor_MorganFP_1017,Descriptor_MorganFP_1018,Descriptor_MorganFP_1019,Descriptor_MorganFP_1020,Descriptor_MorganFP_1021,Descriptor_MorganFP_1022,Descriptor_MorganFP_1023
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3826,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3827,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


If we ever want to use this class again to modify our data, we can recreate it like so:

In [ ]:
from scaffviz.data.dataset import DataSetTSV

DataSetTSV('data/BACE_LIGANDS.tsv').asDataFrame().head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,Descriptor_MorganFP_1016,Descriptor_MorganFP_1017,Descriptor_MorganFP_1018,Descriptor_MorganFP_1019,Descriptor_MorganFP_1020,Descriptor_MorganFP_1021,Descriptor_MorganFP_1022,Descriptor_MorganFP_1023,TSNE_1,TSNE_2
0,AAAGFCUDAXPDAA_on_P56817_WT,High,ChEMBL30,CHEMBL1915983,COCC1CCN(C(C)C(=O)NC(Cc2cc(F)cc(F)c2)C(O)CNCc2...,AAAGFCUDAXPDAA,AAAGFCUDAXPDAA-UHFFFAOYSA-N,InChI=1S/C27H35F2N3O5/c1-17(32-8-7-20(16-36-2)...,"""AuxInfo=1/1/N:9,1,32,34,35,33,5,6,16,22,29,14...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-61.671165,18.636885
1,AACUJFVOHGRMTR_on_P56817_WT,High,ChEMBL30,CHEMBL4204869,CC1OCC2(c3cc(NC(=O)c4cnc(C(F)F)cn4)ccc3F)N=C(N...,AACUJFVOHGRMTR,AACUJFVOHGRMTR-UHFFFAOYSA-N,InChI=1S/C19H18F3N5O2S/c1-9-12-7-30-18(23)27-1...,"""AuxInfo=1/1/N:1,21,22,7,19,13,29,4,2,8,6,30,2...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.981135,-41.491680
2,AANLQBIDWIHDGD_on_P56817_WT,High,ChEMBL30,CHEMBL4159574,COCCOc1cnc(C(=O)Nc2ccc(F)c(C3(C)N=C(N)COC3C(F)...,AANLQBIDWIHDGD,AANLQBIDWIHDGD-UHFFFAOYSA-N,"InChI=1S/C20H21F4N5O4/c1-19(18(20(22,23)24)33-...","""AuxInfo=1/1/N:20,1,14,15,3,4,31,32,7,24,13,18...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,22.383297,-54.882477
3,AARJXZHQDZOZNY_on_P56817_WT,High,ChEMBL30,CHEMBL2443370,CC1=C(C(=O)Nc2nc(C)cs2)C(c2ccco2)C(C(=O)Nc2nc(...,AARJXZHQDZOZNY,AARJXZHQDZOZNY-UHFFFAOYSA-N,InChI=1S/C21H21N5O3S2/c1-10-8-30-20(22-10)25-1...,"""AuxInfo=1/1/N:10,26,1,30,16,15,17,11,27,9,25,...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,26.984573,25.672077
4,AAWSEFDIDJLXJF_on_P56817_WT,High,ChEMBL30,CHEMBL2178717,CN1C(=O)C2CN(c3ncccn3)CC2(c2cc(-c3cccc(C#N)c3)...,AAWSEFDIDJLXJF,AAWSEFDIDJLXJF-UHFFFAOYSA-N,InChI=1S/C22H19N7OS/c1-28-19(30)17-11-29(21-25...,"""AuxInfo=1/1/N:1,21,11,22,20,10,12,26,17,24,6,...",P56817_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-53.715622,-27.205227


## Chemical Space Visualization

Now, we can do our first data analysis of this data set, which will be a visualization of the chemical space covered by the compounds tested against the BACE1 protein in public literature. In order to do that, we will use another feature of `scaffviz`, which is interactive plotting with the [`molplotly`](https://github.com/wjm41/molplotly) package. We can configure the visualization in many different ways, but for now we just want to use the Morgan fingeprints we calculated to calculate a t-SNE embedding to place similar compounds in the same parts of the plot. We also color the points based on the pChEMBL median value extracted from the Papyrus data set. The higher this value, the more affinity/effect the compound should have on the BACE1 protein. Generally, hits with pChEMBL of 6.5 and higher are considered as having significant biological effect:

In [ ]:
from scaffviz.depiction.plot import Plot
from scaffviz.clustering.manifold import TSNE

# make an interactive plot that will use t-SNE to embed the data set in 2D (all current descriptors in the data set will be used)
plt = Plot(dataset, TSNE())

# start the server, you can open the plot in the browser
plt.plot(
    recalculate=False, 
    color_by="pchembl_value_Median",
    color_style="continuous",
    color_continuous_scale="rdylgn",
    card_data=["pchembl_value_Median", "all_doc_ids", "source"],
    title_data="Activity_ID",
    viewport_height=800,
    port=9090
)

<IPython.core.display.Javascript object>

This plot you are seeing is running as a seperate web application embedded into this one. So if you are running this notebook on [localhost:8888](http://localhost:8888), you can navigate to [localhost:9090](http://localhost:9090) to see te full depiction on its own page. You should be able to see clusters of similar molecules and regions of chemical space with active/inactive molecules should be discernible. Therefore, such visualization could be useful to assess how many molecules of what activity we have in the data set and get an idea how similar they are to each other. It is also possible to identify potential activity cliffs (regions where very similar molecules have vastly different activity). 

In the [next part of the tutorial](./qsar.ipynb), we will show a simple worfklow where we use this data to train a simple machine learning model that can learn how to predict whether a compound is likely to be active or inactive on BACE1 protein.